In [5]:
import cv2
import os

In [3]:
def get_video_fps(video_path):
    """
    Returns the frames per second (FPS) of a video file using OpenCV.
    
    Parameters:
        video_path (str): Relative path to the video file
        
    Returns:
        float: The frames per second of the video
        
    Raises:
        ValueError: If the video file cannot be opened or is invalid
    """
    try:
        # Create a VideoCapture object
        video = cv2.VideoCapture(video_path)
        
        # Check if video file was successfully opened
        if not video.isOpened():
            raise ValueError(f"Error: Could not open video file at {video_path}")
        
        # Get the frames per second property
        # CV_CAP_PROP_FPS is the property identifier for FPS in OpenCV
        fps = video.get(cv2.CAP_PROP_FPS)
        
        # Release the video capture object
        video.release()
        
        # Some video files might return 0 for FPS, which indicates an error
        if fps <= 0:
            raise ValueError(f"Error: Could not determine FPS for video at {video_path}")
            
        return fps
        
    except Exception as e:
        # Catch any other exceptions that might occur
        raise ValueError(f"Error processing video: {str(e)}")

In [4]:
video_path = "./youtube_DNViaspA8hM_1920x1080_h264.mp4"
fps = get_video_fps(video_path=video_path)
print(fps)

29.97002997002997


In [6]:
def resample_video_fps(video_path, desired_fps):
    """
    Resamples a video to a desired frames per second (FPS) rate by uniformly
    sampling frames from the original video.
    
    Parameters:
        video_path (str): Relative path to the input video file
        desired_fps (float): The desired FPS for the output video
        
    Returns:
        str: Path to the resampled video
        
    Raises:
        ValueError: If there's an issue processing the video
    """
    try:
        # Get the current FPS of the video using the previously defined function
        current_fps = get_video_fps(video_path)
        
        # Create a VideoCapture object to read the input video
        cap = cv2.VideoCapture(video_path)
        
        # Check if video file was successfully opened
        if not cap.isOpened():
            raise ValueError(f"Error: Could not open video file at {video_path}")
        
        # Get the video properties we need for processing
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # Calculate the total duration of the video in seconds
        duration = total_frames / current_fps
        
        # Calculate how many frames will be in the output video
        # based on the desired FPS and the original duration
        output_total_frames = int(duration * desired_fps)
        
        # Generate the output video path in the same directory as the input
        # with a suffix indicating the new FPS
        video_dir = os.path.dirname(video_path)
        video_filename = os.path.basename(video_path)
        base_name, ext = os.path.splitext(video_filename)
        output_path = os.path.join(video_dir, f"{base_name}_fps{int(desired_fps)}{ext}")
        
        # Create a VideoWriter object to write the output video
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4
        out = cv2.VideoWriter(output_path, fourcc, desired_fps, (width, height))
        
        # For each frame in the output video, calculate and retrieve
        # the corresponding frame from the input video
        for i in range(output_total_frames):
            # Calculate which frame to use from the original video
            # using the formula: round(i*current_fps/desired_fps)
            original_frame_idx = round(i * current_fps / desired_fps)
            
            # Ensure we don't go beyond the total frames in the original video
            if original_frame_idx >= total_frames:
                break
                
            # Set the video capture to the calculated frame position
            cap.set(cv2.CAP_PROP_POS_FRAMES, original_frame_idx)
            
            # Read the frame from the input video
            ret, frame = cap.read()
            if not ret:
                # If we can't read the frame, stop processing
                break
                
            # Write the frame to the output video
            out.write(frame)
        
        # Release the video capture and writer objects to free resources
        cap.release()
        out.release()
        
        # Return the path to the newly created video
        return output_path
        
    except Exception as e:
        # Catch any other exceptions that might occur
        raise ValueError(f"Error resampling video: {str(e)}")

In [8]:
desired_fps=10
output_path = resample_video_fps(video_path=video_path, desired_fps=desired_fps)

In [9]:
def count_frames(video_path):
    """
    Counts the number of frames in a video file using OpenCV.
    
    Parameters:
        video_path (str): Path to the video file
        
    Returns:
        int: The number of frames in the video
        
    Raises:
        ValueError: If the video file cannot be opened or is invalid
    """
    try:
        # Create a VideoCapture object
        video = cv2.VideoCapture(video_path)
        
        # Check if video file was successfully opened
        if not video.isOpened():
            raise ValueError(f"Error: Could not open video file at {video_path}")
        
        # Method 1: Get frame count using the property
        # This is fast but can be inaccurate for some video formats
        frame_count_prop = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # Method 2: Actually count frames (more accurate but slower)
        # Uncomment this block for more accurate counting if needed
        """
        frame_count_actual = 0
        video.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reset to beginning
        while True:
            ret, _ = video.read()
            if not ret:
                break
            frame_count_actual += 1
        print(f"Actual frame count: {frame_count_actual}")
        """
        
        # Release the video capture object
        video.release()
        
        return frame_count_prop
        
    except Exception as e:
        # Catch any other exceptions that might occur
        raise ValueError(f"Error counting frames: {str(e)}")

In [12]:
frames_original = count_frames(video_path)
frames_sampled = count_frames(output_path)
print(frames_original, frames_sampled)

1955 652


In [17]:
for sample_fps in range(12, 26, 2):
    output_path_1 = resample_video_fps(video_path=video_path, desired_fps=sample_fps)
    print(f"Made video on {sample_fps}")


Made video on 12
Made video on 14
Made video on 16
Made video on 18
Made video on 20
Made video on 22
Made video on 24
